
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [34]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.
import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split

df = pd.read_csv('data/Corona_NLP_train.csv', encoding='latin1')

#Drop all rows in df with neutral sentiment
df = df[df['Sentiment'] != 'Neutral']

X = df['OriginalTweet']
X = X.tolist()
X = np.array(X)
print(type(X))

# List comprehension
Y = [1 if sentiment in ['Positive', 'Extremely Positive'] else 0 for sentiment in df['Sentiment']] # else None for sentiment in df['Sentiment']]
Y = np.array(Y)
#X_train = X[:int(len(X)*0.60)] # 65 %
#Y_train = Y[:int(len(Y)*0.60)]

#X_test = X[int(len(X)*0.60):int(len(X)*0.75)] # 15 %
#Y_test = Y[int(len(Y)*0.60):int(len(Y)*0.75)]

#X_valid = X[int(len(X)*0.75):]
#Y_valid = Y[int(len(X)*0.75):]

# I Think the auto grader likes this way better, because it rounds in the same way the answer is
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, train_size=0.6, shuffle=False)

X_test, X_valid, Y_test, Y_valid = train_test_split(X_temp, Y_temp, train_size=0.375, shuffle=False)


print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

print(X)

<class 'numpy.ndarray'>
(20066,)
(5016,)
(8362,)
['advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order'
 'Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P'
 "My food stock is not the only one which is empty...\r\n\r\n\r\n\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\n\r\nStay calm, stay safe.\r\n\r\n\r\n\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j"
 ...
 'Response to complaint not provided citing COVID-19 related delays. Yet prompt in rejecting policy before consumer TAT is over. Way to go ?'
 'You know itÂ\x92s getting tough when @KameronWilds  is rationing toilet paper #coronavirus 

In [35]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

#Convert X using `CountVectorizer`as the first step. The step that comes after should be a `LogisticRegression` model, 
# but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`]
# Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that
#  `model.predict_proba` works for this. 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
#vectorizer = CountVectorizer(encoding='latin1')
#vectorizer.fit(X_train)
#print(vectorizer)
#print("Vocabulary: ", vectorizer.vocabulary_) # prints unique words and indices
#encode
#vector = vectorizer.transform(X_train)

# Summarizing the Encoded Texts
#print("Encoded Document is:")
#print(vector.toarray())


# verbose outputs messages for debugging. Added max_iter=1000 because default is 100 and gave a warning.
model = Pipeline([('vectorizer', CountVectorizer(encoding='latin1')), ('logistic_regression', LogisticRegression(solver='liblinear', max_iter=1000))], verbose = True)

# fitting the data in the pipe
model.fit(X_train, Y_train) # Automatically handles fitting of countvectorizer and logistic regression
y_pred = model.predict(X_test) # Works, at least prints.
model.predict_proba(X_test) # Works, it at least prints.

# Test predict_proba functionality
proba = model.predict_proba(X_test[:5])
print("Predicted probabilities for the first 5 test samples:")
print(proba)

[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=   0.6s
[Pipeline]  (step 2 of 2) Processing logistic_regression, total=   1.8s
Predicted probabilities for the first 5 test samples:
[[0.87766879 0.12233121]
 [0.99712779 0.00287221]
 [0.03468536 0.96531464]
 [0.7988057  0.2011943 ]
 [0.43882767 0.56117233]]


In [36]:

# Part 3
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

# y_true, y_pred
precision_0 = precision_score(Y_test, y_pred, pos_label=0)
print(f"precision for 0: {precision_0}")
precision_1 = precision_score(Y_test, y_pred, pos_label=1)
print(f"precision for 1: {precision_1}")

recall_0 = recall_score(Y_test, y_pred, pos_label=0)
print(f"recall for 0: {recall_0}")
recall_1 = recall_score(Y_test, y_pred, pos_label=1)
print(f"recall for 1: {recall_1}")

precision for 0: 0.8579595426561126
precision for 1: 0.8708971553610503
recall for 0: 0.8464208242950109
recall for 1: 0.8808557727775729


In [37]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost

    #4. [3p] Let us now define a cost function
    #* A positive tweet that is classified as negative will have a cost of 1
    #* A negative tweet that is classified as positive will have a cost of 5
    #* Correct classifications cost 0
   # 
   # complete filling the function `cost` to compute the cost of a prediction model under a certain prediction 
   # threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

    y_pred_proba1 = model.predict_proba(X)[:, 1] # probs for class 1 POSITIVE

    #print(model.classes_) # To see which order classes are in

    # bayes classifier rule says 1 if r(x) > 1/2, else 0
    # Assume that the model is fitted


    # does same as following loop
    #y_pred = (y_pred_proba1 >= threshold).astype(int) 

    #y_pred = []
    #for i in range(len(y_pred_proba1)):
    #    if y_pred_proba1[i] >= threshold: # if prob that 1 is predicted is at least threshold
    #        y_pred.append(1)
    #    else:
    #        y_pred.append(0)

    predictions = (y_pred_proba1 >= threshold).astype(int)
    # Where Y == 1, put the boolean value of y_pred == 0 but make into int (1 true 0 false), same if y_pred == 1 but take times 5
    cost_list = np.where(Y == 1, (predictions == 0).astype(int), (predictions == 1).astype(int) * 5)
    cost_list 
    cost_avg = np.mean(cost_list)
    return cost_avg



    #cost_list = []
    #for i in range (len(Y)):
    #    if Y[i] == 1: # if positive tweet
    #        if y_pred[i] == 1:
    #            cost_list.append(0)
    #        else:
    #            cost_list.append(1)
    #    if Y[i] == 0: # If negative tweet
    #        if y_pred[i] == 0:
    #            cost_list.append(0)
    #        else:
    #            cost_list.append(5)
    #cost_avg = sum(cost_list)/len(Y) # Divide by length of list




print("cost: " + str(cost(model, 0.3, X, Y))) # Average cost 2.167 for threshold 0.5, 




cost: 0.38120440138739387


In [38]:

# Part 5

# Generate thresholds from 0.6 to 1.0 with 0.01 increments

thresholds = np.arange(0.7, 1.01, 0.01) #np.linspace(0, 1, 101)# # linspace same 

print("done")

lowest = 100
best_threshold1 = 0
for threshold1 in thresholds:
    current = cost(model, threshold1, X_test, Y_test)
    #print(f"current: {current}")
    if current < lowest:
        lowest = current
        best_threshold1 = threshold1
    #print(f"lowest cost: {lowest}. Best threshold: {best_threshold1}")

print(lowest)
print(best_threshold1)

# lowest cost: 0.2727272727272727. Best threshold: 0.8100000000000002
optimal_threshold = best_threshold1#0.8100000000000
cost_at_optimal_threshold = lowest# 0.2727272727272727

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

done
0.2727272727272727
0.8


In [39]:

# Part 6

#6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing 
# the cost using the validation data. 
# Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
print(f"optimal cost {cost_at_optimal_threshold_valid}")
# remember to take cost range into account
cost_range = 5 # Since cost can be from 0 - 5, multiply epsilon by range (5)
epsilon = np.sqrt(np.log(2/0.01) / (2 * len(Y_valid))) * cost_range
print(f"epsilon {epsilon}")
epsilon_wo_range = np.sqrt(np.log(2/0.01) / (2 * len(Y_valid)))
cost_interval_valid = cost_at_optimal_threshold_valid - epsilon, cost_at_optimal_threshold_valid + epsilon
print(f"Cost with range (correct): {cost_interval_valid}")
print(f"Cost without range: {cost_at_optimal_threshold_valid - epsilon_wo_range, cost_at_optimal_threshold_valid + epsilon_wo_range}")
assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

optimal cost 0.27744558718010043
epsilon 0.08899567661998982
Cost with range (correct): (np.float64(0.18844991056011062), np.float64(0.36644126380009023))
Cost without range: (np.float64(0.25964645185610247), np.float64(0.2952447225040984))


In [40]:

# Part 7

def cost_random_tweet(model, threshold, X, Y):
    probabilities = model.predict_proba(X)[:, 1] # probability for a tweet to be positive
    false_negatives = (1 - (probabilities >= threshold).astype(int)) * Y
    false_positives = 5 * (1-Y) * (probabilities >= threshold).astype(int)
    C = false_negatives + false_positives
    return C

# C for validation set
C_valid = cost_random_tweet(model, optimal_threshold, X_valid, Y_valid)

# Variance
# Without ddof its population var 
# (if data is not just sample), with ddof its sample variance.
variance_of_C = np.var(C_valid, ddof=1)  

# Bennett's inequality
# gives "probabilistic bound on the deviation of a sample mean from the true mean"
# considers both variance and sample mean
num_validation_samples = len(C_valid)
confidence = 0.99
delta = 1 - confidence
# bennet's inequality formula
# sqrt(2 * var * ln(1/delta) / n)
epsilon = np.sqrt((2 * variance_of_C * np.log(1 / delta)) / num_validation_samples)

C_mean= np.mean(C_valid)

interval_of_C = C_mean - epsilon, C_mean + epsilon

print(interval_of_C)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

(np.float64(0.2502933787639524), np.float64(0.30459779559624844))
